## Final Project Notebook

The report for this final project can be found at this [link](https://github.com/cybertraining-dsc/fa20-523-301/blob/master/project/project.md).

## Part 1 Importing the functions

This file requires that we import Numpy, Matplotlib, Pylab, Keras, and Pandas

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

A link for Keras for us to use can be found [here](https://keras.io/guides/sequential_model/).

In [2]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)
# Call model on a test input
x = tf.ones((3, 3))
y = model(x)

Insert Layers

In [3]:
# Create 3 layers
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# Call layers on a test input
x = tf.ones((3, 3))
y = layer3(layer2(layer1(x)))